In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mlds

Mounted at /content/drive
/content/drive/MyDrive/mlds


In [2]:
import data_bitcoin as btc
import tasker_link_prediction as t_lp
from splitter import splitter
from models import Baseline_node2vec
from trainer import Trainer
import yaml
import os

In [3]:
prep = False

with open('config/config_BTC_Alpha_baseline.yaml', 'r') as file:
    config = yaml.safe_load(file)

model_path = config['model_path']

if not os.path.exists(model_path):
    os.mkdir(model_path)
    os.mkdir(model_path + "best_checkpoints/")
    os.mkdir(model_path + "latest_checkpoints/")

with open(model_path + 'config.yaml', 'w') as file:
    yaml.dump(config, file)

config

{'adam_config': {'adam_beta_1': 0.9,
  'adam_beta_2': 0.999,
  'adam_epsilon': 1e-07,
  'adam_learning_rate': 0.005},
 'model_path': 'models/BTC_ALPHA_N2V_baseline/',
 'classifier_hidden_size': 16,
 'ffn_fusion_size': 8,
 'ffn_hidden_size': 8,
 'gcn_fusion_size': 16,
 'spatial_hidden_size': 16,
 'spatial_input_dim': 128,
 'temporal_hidden_size': 16,
 'temporal_input_dim': 100,
 'num_epochs': 1000,
 'patience': 10,
 'major_threshold': None,
 'train_proportion': 0.7,
 'dev_proportion': 0.1,
 'data_path': 'data/BTC-ALPHA/',
 'prep_data_path': 'prep_data/BTC_Alpha_neg/'}

In [4]:
data = btc.Bitcoin_Dataset(config['data_path'])
tasker = t_lp.Link_Pred_Tasker(data, path=config['prep_data_path'], prep=prep,
                               embs_dim=config['spatial_input_dim'], temp_dim=int(config['temporal_input_dim']/10),
                               major_threshold=config['major_threshold'], smart_neg_sampling=True)

splitter_as = splitter(tasker, train_proportion = config['train_proportion'], dev_proportion = config['dev_proportion'])

model= Baseline_node2vec(config['spatial_input_dim'])

trainer = Trainer(model=model, splitter=splitter_as, model_path=model_path, adam_config=config['adam_config'], patience=config['patience'])

Dataset splits sizes:  train 85 dev 14 test 28


In [5]:
trainer.train(config['num_epochs'])

Epoch 1, Step 1, Loss: 0.08404892683029175
Epoch 1, Step 2, Loss: 0.0722685232758522
Epoch 1, Step 3, Loss: 0.061451930552721024
Epoch 1, Step 4, Loss: 0.043301522731781006
Epoch 1, Step 5, Loss: 0.03838137164711952
Epoch 1, Step 6, Loss: 0.030119020491838455
Epoch 1, Step 7, Loss: 0.023640932515263557
Epoch 1, Step 8, Loss: 0.03127807378768921
Epoch 1, Step 9, Loss: 0.0333527997136116
Epoch 1, Step 10, Loss: 0.0329669751226902
Epoch 1, Step 11, Loss: 0.028964119032025337
Epoch 1, Step 12, Loss: 0.02888542227447033
Epoch 1, Step 13, Loss: 0.02659381553530693
Epoch 1, Step 14, Loss: 0.027989663183689117
Epoch 1, Step 15, Loss: 0.025209294632077217
Epoch 1, Step 16, Loss: 0.027382254600524902
Epoch 1, Step 17, Loss: 0.03062332607805729
Epoch 1, Step 18, Loss: 0.03478565439581871
Epoch 1, Step 19, Loss: 0.032713133841753006
Epoch 1, Step 20, Loss: 0.034327369183301926
Epoch 1, Step 21, Loss: 0.027364380657672882
Epoch 1, Step 22, Loss: 0.023812005296349525
Epoch 1, Step 23, Loss: 0.026961

In [6]:
trainer.restore_best_checkpoint()
trainer.validate('Validation')
trainer.validate('Test')

Validation Metric MRRs: 0.06248399057505962
Validation Metric MAPs: 0.01478725100262871
Test Metric MRRs: 0.09344134729319453
Test Metric MAPs: 0.03250185273997393


(0.09344134729319453, 0.03250185273997393)